<a href="https://colab.research.google.com/github/sizhky/AutoTrain/blob/master/nbs/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup Data and Libraries

In [1]:
%%capture
# Setup imports
try:
    import timm
except:
    !pip install -U torch-snippets typer imgaug timm fastai

In [2]:
# Setup Data
%load_ext autoreload
%autoreload 2

from torch_snippets import *
from fastai.vision.all import *

# If running locally from AutoTrain/nbs
if not os.path.exists('/content') and P().resolve().stem != 'AutoTrain':
    %cd ..

# If running from colab
if os.path.exists('/content') and not os.path.exists('/content/AutoTrain'):
    !git clone https://github.com/sizhky/AutoTrain

# change present working directory
if not P().resolve().stem == 'AutoTrain':
    %cd AutoTrain
    
sys.path.append(str(P().resolve()))

# !git pull origin enhancement/fastai-classification
# !git checkout enhancement/fastai-classification

if not os.path.exists('imagenette2-160'):
    o = untar_data(URLs.IMAGENETTE_160)
    !cp -r {o} ./


2021-09-28 23:01:19.871 | WARNING  | torch_snippets:<module>:13 - sklearn is not found. Skipping relevant imports from submodule `sklegos`
Exception: No module named 'sklego'


/mnt/sda1/code/AutoTrain


## Verify Config

In [3]:
!cat configs/classification_imagenette.ini

[project]
owner: sizhky
name: imagenette
location: ${project.name}
model_directory: ${project.location}/models
type: classification
num_classes: 10

[architecture]
image_size: 224
[architecture.backbone]
model: efficientnet_b0
# everytime you change the model, vector size 
# must match the model's backbone output.
vector_size: 1280
[architecture.head]
@head: custom_head
head_input_size: ${architecture.backbone.vector_size}
num_classes: ${project.num_classes}

[training]
dir: imagenette2-160
train_ratio: 0.8
[training.preprocess]
@preprocess_function: my_preprocess
image_size: ${architecture.image_size}

[training.data]
train_dir: ${training.dir}/train
validation_dir: ${training.dir}/validation

[training.scheme]
epochs: 2
freeze_epochs: 2
# initial_checkpoint: ${online_training.save_dir}
# mixup augmentation
mixup: 0.5
# cutmix
cutmix: 0.5 
mixup_switch_prob: 0.3
amp: true
# random augmentations
aa: rand-m9-mstd0.5
batch_size: 2
output: ${project.model_directory}/trained-model-v1
outpu

In [11]:
import os; os.environ['CONFIG'] = 'configs/classification_imagenette.ini'
!python auto_train/classification/train.py

2021-09-28 22:31:45.654 | WARNING  | torch_snippets:<module>:13 - sklearn is not found. Skipping relevant imports from submodule `sklegos`
Exception: No module named 'sklego'
2021-09-28 22:31:50.167 | INFO     | __main__:train_model:35 - Training from scratch!
2021-09-28 22:33:10.323 | INFO     | __main__:find_best_learning_rate:22 - LR Plot is saved at imagenette/find_lr_plot.png
2021-09-28 22:33:10.324 | INFO     | __main__:train_model:40 - Using lr: 0.00019054606673307717
['Epoch, Train Loss, Validation Loss, Validation Accuracy, Time']
[0, 1.7711164951324463, None, None, '00:59']
[1, 0.8143174052238464, None, None, '00:15']
[0, 0.627459704875946, None, None, '00:18']
[1, 0.5707716941833496, None, None, '00:18']
2021-09-28 22:35:03.645 | INFO     | __main__:train_model:50 - Saved weights of size ~18 MB to imagenette/models/trained-model-v1/model.pth


In [6]:
import os; os.environ['CONFIG'] = 'configs/classification_imagenette.ini'
!python main.py infer --task=classification --folder='imagenette2-160/val/n02102040/'

2021-09-28 23:06:29.226 | WARNING  | torch_snippets:<module>:13 - sklearn is not found. Skipping relevant imports from submodule `sklegos`
Exception: No module named 'sklego'
2021-09-28 23:06:34.955 | INFO     | auto_train.classification.infer:<module>:6 - Loaded weights from imagenette/models/trained-model-v1/model.pth to given model
2021-09-28 23:06:35.318 | DEBUG    | __main__:infer:31 - 
[
    {
        "file": "/mnt/sda1/code/AutoTrain/imagenette2-160/val/n02102040/n02102040_7452.JPEG",
        "prediction": "n02102040",
        "confidence": "87.61%"
    },
    {
        "file": "/mnt/sda1/code/AutoTrain/imagenette2-160/val/n02102040/n02102040_1782.JPEG",
        "prediction": "n02102040",
        "confidence": "69.71%"
    },
    {
        "file": "/mnt/sda1/code/AutoTrain/imagenette2-160/val/n02102040/n02102040_390.JPEG",
        "prediction": "n02102040",
        "confidence": "80.91%"
    },
    {
        "file": "/mnt/sda1/code/AutoTrain/imagenette2-160/val/n02102040/n021020